# Getting started with Delta Lake

This notebook will guide you with basics operations.

**_Delta Lake_** is an open-source storage layer that brings ACID to Apache Spark and big data workloads. With Delta Lake providing atomic transactionality for your data, this allows you - the data practitioner - to need to only focus on building your data processing pipelines by expressing the processing

Let’s create our first Delta table! Like databases, to create our Delta table we can first create a table definition and define the schema or, like data lakes simply write a Spark DataFrame to storage in the Delta format.


> **Side Note:** 
> What is a metastore?

> Fabric will make a table definition in a metastore (commonly this is a Hive metastore). This table definition is a metadata entry that will describe to data processing frameworks such as Apache Spark the data location, storage format, table schema, as well as other properties.

In [ ]:
%%sql 
SELECT current_database();

## Managed vs Unmanaged table 

Delta Lake supports creating two types of tables — tables defined in the metastore **(managed)** and tables defined by path **(external/unmanaged)**.

## Create managed Delta table

> **DeltaTableBuilder API**: You can also use the DeltaTableBuilder API in Delta Lake to create tables. Compared to the DataFrameWriter APIs, this API makes it easier to specify additional information like column comments, table properties, and generated columns

In [ ]:
spark.sql("DROP TABLE IF EXISTS demo.myTable") 

In [ ]:
import delta

delta.DeltaTable.create(spark) \
  .tableName("myTable") \
  .addColumn("id", "LONG") \
  .addColumn("name", "STRING") \
  .execute() 

> **DataFrameWriter API**: If you want to simultaneously create a table and insert data into it from Spark DataFrames or Datasets, you can use the Spark DataFrameWriter

In [ ]:
spark.sql("DROP TABLE IF EXISTS myTable")
# Create data DataFrame
data = spark.range(0, 10)
# Write the data DataFrame to onelake location
data.write.format("delta").save("Tables/myTable")

> In addiotion, you can use _**saveAsTable**_

In [ ]:
spark.sql("DROP TABLE IF EXISTS demo.myTable")
# Create data DataFrame
data = spark.range(0, 10)
# Write the data DataFrame to onelake location
data.write.format("delta").saveAsTable("myTable") 

> **SQL DDL commands**: You can use standard SQL DDL commands supported in Apache Spark (for example, CREATE TABLE and REPLACE TABLE) to create Delta tables

In [ ]:
%%sql
DROP TABLE IF EXISTS myTable; 

CREATE TABLE myTable (
id BIGINT,
name STRING)
USING DELTA 

## Create unmanaged table

> **DeltaTableBuilder API**: You can also use the DeltaTableBuilder API in Delta Lake to create tables. Compared to the DataFrameWriter APIs, this API makes it easier to specify additional information like column comments, table properties, and generated columns

In [ ]:
import delta

spark.sql("DROP TABLE IF EXISTS myExternalTable")

delta.DeltaTable.create(spark) \
  .tableName("myExternalTable") \
  .addColumn("id", "LONG") \
  .location("Files/myExternalTable") \
  .execute()

> **DataFrameWriter API**: If you want to simultaneously create a table and insert data into it from Spark DataFrames or Datasets, you can use the Spark DataFrameWriter

In [ ]:
# Create data DataFrame
data = spark.range(0, 5)
# Write the data DataFrame to onelake location
data.write.format("delta").mode("append").saveAsTable("myExternalTable") 

At this moment, you can see in the Lakehouse explorer under Files that a **new folder called myExternalTable** was created.

In [ ]:
%%sql
SELECT * FROM demo.myExternalTable

> Let's **drop the table!** 

In [ ]:
%%sql
DROP TABLE IF EXISTS demo.myExternalTable 

Refresh the Lakehouse explorer and you will keep seeing myExternalTable under Files but not in the Tables.

However, let's create the again and will see that all our data is still there! 

In [ ]:
%%sql
CREATE TABLE myExternalTable
(id LONG)
USING DELTA
LOCATION 'Files/myExternalTable' 


In [ ]:
%%sql
SELECT * FROM myExternalTable 

## Partitioned Table

In [ ]:
%%sql
CREATE TABLE myTable_partitioned ( action STRING, date TIMESTAMP, device_id INT) 
USING DELTA 
PARTITIONED BY (date) 

> OR

In [ ]:
from pyspark.sql.functions import expr, lit, col
from pyspark.sql.types import *
from datetime import date

spark.sql("DROP TABLE IF EXISTS mytable_partitioned")

df = spark.range(5) \
  .selectExpr("if(id % 2 = 0, 'Open', 'Close') as action") \
  .withColumn("date", expr("cast(concat('2023-06-', cast(rand(5) * 30 as int) + 1) as date)")) \
  .withColumn("device_id", expr("cast(rand(5) * 100 as int)"))

df.write.format("delta").partitionBy('date').saveAsTable("mytable_partitioned") 

Let's check how the data is organized using the lakehouse explorer

## Working with Delta table

A DeltaTable is the entry point for interacting with tables programmatically in Python — for example, to perform updates or deletes and so on.

You can use either table name or table path.

In [ ]:
from delta.tables import *

# using table name
myTable = DeltaTable.forName(spark, "myTable")
# using table path
myExternalTable = DeltaTable.forPath(spark,"Files/myExternalTable")

## Writing to Delta table

### Append

In [ ]:
# Create data DataFrame
data = spark.range(10, 15).withColumn('name', lit('Michael'))
# Write the data DataFrame to onelake location
data.write.mode("append").format("delta").saveAsTable("mytable")

In [ ]:
%%sql
SELECT * FROM demo.mytable

In [ ]:
%%sql
INSERT INTO demo.myTable VALUES (16,'Michael');
INSERT INTO demo.myTable VALUES (17,'Michael');
INSERT INTO demo.myTable VALUES (18,'Michael');

### Overwrite

In [ ]:
# Create data DataFrame
data = spark.range(0, 11).withColumn('name', lit('Michael'))
# Write the data DataFrame to onelake location
data.write.mode("overwrite").format("delta").saveAsTable("mytable")

In [ ]:
%%sql
SELECT * FROM demo.mytable

In [ ]:
%%sql
INSERT OVERWRITE demo.myTable SELECT 1, 'Michael';

In [ ]:
%%sql
SELECT * FROM demo.mytable

## Delete

Don't need to get the delta table info again! Just for demo purposes.

In [ ]:
spark.sql("DROP TABLE IF EXISTS demo.myTable")
# Create data DataFrame
data = spark.range(0, 15)
# Write the data DataFrame to onelake location
data.write.format("delta").mode("append").saveAsTable("myTable") 

In [ ]:
%%sql
SELECT * FROM demo.mytable

In [ ]:
from delta.tables import * 
from pyspark.sql.functions import *

deltaTable = DeltaTable.forName(spark, 'myTable')  

deltaTable.delete(col('id') > 10)

> OR

In [ ]:
%%sql

DELETE FROM demo.myTable WHERE ID > 10;

In [ ]:
%%sql
SELECT * FROM demo.mytable

## Update

Don't need to get the delta table info again! Just for demo purposes.

In [ ]:
from delta.tables import * 
deltaTable = DeltaTable.forName(spark, 'myTable')  

deltaTable.update(
  condition = expr("id % 2 == 0"),
  set = { "id": expr("id + 100") }
)


In [ ]:
%%sql
SELECT * FROM demo.mytable

In [ ]:
%%sql

UPDATE demo.myTable 
SET id = id + 100 
WHERE id % 2 = 0

## Merge

Don't need to get the delta table info again! Just for demo purposes.

In [ ]:
from delta.tables import *

deltaTable = DeltaTable.forName(spark, 'mytable')  

newdata = spark.range(0, 10)

deltaTable.alias('my_table') \
    .merge( newdata.alias('new_data'), 'my_table.id = new_data.id') \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

> Explicitly using columns

In [ ]:
newdata = spark.range(0, 10)

deltaTable.alias('my_table') \
    .merge( newdata.alias('new_data'), 'my_table.id = new_data.id') \
    .whenMatchedUpdate(set = {"id": "new_data.id"}) \
    .whenNotMatchedInsert(values = {"id": "new_data.id"}) \
    .execute()

In [ ]:
%%sql
SELECT * FROM demo.mytable

> OR

In [ ]:
newdata.createOrReplaceTempView('newData')

In [ ]:
%%sql
MERGE INTO demo.myTable
USING newData
ON myTable.id = newData.id
WHEN MATCHED THEN
  UPDATE SET
    id = newData.id
WHEN NOT MATCHED
  THEN INSERT (
    id
  )
  VALUES (
    newData.id
  )

## Reading from Delta table

In [ ]:
myTable_df = spark.read.table("myTable") 
display(myTable_df)

> OR

In [ ]:
myTable_df = spark.read.format("delta").load("Tables/mytable")
display(myTable_df.limit(5))


In [ ]:
%%sql
SELECT * FROM demo.myTable LIMIT 5

## Retrieve table detail

Don't need to import and get the delta table info again! Just for demo purposes.

In [ ]:
from delta.tables import * 
deltaTable = DeltaTable.forName(spark, 'myTable')
 
detailsDF = deltaTable.detail() 
display(detailsDF)

In [ ]:
%%sql
DESCRIBE DETAIL demo.myTable

In [ ]:
%%sql
DESCRIBE EXTENDED demo.myTable

In [ ]:
from delta.tables import * 
deltaTable = DeltaTable.forPath(spark, 'Files/myExternalTable')
 
detailsDF = deltaTable.detail() 
display(detailsDF)

In [ ]:
%%sql
DESCRIBE DETAIL demo.myExternalTable

In [ ]:
%%sql
DESCRIBE EXTENDED demo.myExternalTable

## Create table LIKE Command

CREATE TABLE LIKE makes it easy to build a new table with the same columns and schema as an existing table.

Let's create a Delta table and then create another table with the same schema.

In [ ]:
%%sql
CREATE TABLE demo.mytable_duplicate LIKE demo.mytable

In [ ]:
%%sql
DESCRIBE EXTENDED demo.mytable_duplicate

# Clean up

In [ ]:
spark.sql("DROP TABLE IF EXISTS demo.mytable") 
spark.sql("DROP TABLE IF EXISTS demo.myExternalTable")
spark.sql("DROP TABLE IF EXISTS demo.mytable_partitioned")
spark.sql("DROP TABLE IF EXISTS demo.mytable_duplicate")

> OR

In [ ]:
%%sql
DROP TABLE demo.myTable;

In [ ]:
if mssparkutils.fs.exists("Files/myExternalTable"):
    mssparkutils.fs.rm("Files/myExternalTable", True)